# Decision Tree & kNN

Performed on original dataset

#### Importing libraries


In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Load Train and Test set

In [2]:
df_train = pd.read_csv('csv/train.csv')
df_test = pd.read_csv('csv/test.csv')

#### Preparing data for model

* X_train-> features of the training set
* X_test ->features of the test set
* y_train -> target variables of the training set
* y_test ->target variables of the test set

NB. Dropped "Subject"

In [3]:
X_train = df_train.drop(['subject','Activity','ActivityName'], axis = 1)
y_train = df_train.ActivityName

In [4]:
X_test = df_test.drop(['subject','Activity','ActivityName'], axis = 1)
y_test = df_test.ActivityName

In [5]:
X_train.shape, y_train.shape

((7352, 561), (7352,))

In [6]:
X_test.shape, y_test.shape

((2947, 561), (2947,))

In [7]:
labels=['LAYING', 'SITTING','STANDING','WALKING','WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']

# K Nearest Neighbor

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import svm, datasets
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall
from scikitplot.metrics import plot_cumulative_gain
from scikitplot.metrics import plot_lift_curve

In [9]:
# Pre-processing kNN
parameters = {'n_neighbors': np.arange(20, 100 + 1), 'weights':['uniform', 'distance']}

clf_knn = KNeighborsClassifier()
search = GridSearchCV(clf_knn, parameters)
search.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100]),
                         'weights': ['uniform', 'distance']})

In [10]:
search.best_estimator_

KNeighborsClassifier(n_neighbors=20, weights='distance')

In [11]:
# Test 1: n_neighbors=20, weights='distance'
clf_knn = KNeighborsClassifier(n_neighbors=20, weights='distance')
clf_knn.fit(X_train, y_train)
y_pred = clf_knn.predict(X_test)
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.9087207329487614
F1-score [0.99813433 0.86120219 0.89105403 0.91401869 0.86465177 0.90390707]
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.93      0.80      0.86       491
          STANDING       0.84      0.95      0.89       532
           WALKING       0.85      0.99      0.91       496
WALKING_DOWNSTAIRS       0.96      0.78      0.86       420
  WALKING_UPSTAIRS       0.90      0.91      0.90       471

          accuracy                           0.91      2947
         macro avg       0.91      0.90      0.91      2947
      weighted avg       0.91      0.91      0.91      2947



In [12]:
result = confusion_matrix(y_test,y_pred)
print('Confusion matrix kNN:')
print(result)

Confusion matrix kNN:
[[535   1   1   0   0   0]
 [  0 394  93   0   0   4]
 [  0  29 503   0   0   0]
 [  0   0   0 489   7   0]
 [  0   0   0  47 329  44]
 [  0   0   0  38   5 428]]


# Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pydotplus
from IPython.display import Image 

In [17]:
clf_dt = DecisionTreeClassifier()
clf_dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [20]:
# Hyperparameters tuning RandomizedSearchCV
param_list = {"criterion": ['gini', 'entropy'], # impurity measurement 
              'max_depth': [None] + [2, 3, 4, 6, 8, 10], # cut of the leaf
             'min_samples_split': [2, 3, 4, 5, 8, 10], # min n. of samples required in a node to split the node
             'min_samples_leaf': [1, 2, 3, 4, 5, 10] # min n. of samples required to be in both left and right 
                                                                         # child of a parent node to split that parent node
             }
random_search = RandomizedSearchCV(clf_dt, param_distributions=param_list, scoring='recall', n_iter=50, random_state=42)
random_search.fit(X_train, y_train)

res = random_search.cv_results_

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

...

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1872, in recall_score
    _, r, _, _ = precision_recall_fscore_support(
  File "C:\Users\Cosimo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1534, in precision_recall_fscore_supp

In [21]:
random_search.best_estimator_

DecisionTreeClassifier(max_depth=6, min_samples_leaf=5, min_samples_split=10)

In [22]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [23]:
report(res, n_top=3)

Model with rank: 1
Mean validation score: nan (std: nan)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 5, 'max_depth': 6, 'criterion': 'gini'}

Model with rank: 2
Mean validation score: nan (std: nan)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 2, 'criterion': 'entropy'}

Model with rank: 3
Mean validation score: nan (std: nan)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 8, 'criterion': 'gini'}



In [24]:
clf_dt = DecisionTreeClassifier(criterion='gini', max_depth=6, min_samples_leaf=5, min_samples_split=10)
clf_dt.fit(X_train, y_train)

y_pred = clf_dt.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.8554462164913471
F1-score [1.         0.78221209 0.83661249 0.89776047 0.78822197 0.79398148]
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.89      0.70      0.78       491
          STANDING       0.77      0.92      0.84       532
           WALKING       0.87      0.93      0.90       496
WALKING_DOWNSTAIRS       0.75      0.83      0.79       420
  WALKING_UPSTAIRS       0.87      0.73      0.79       471

          accuracy                           0.86      2947
         macro avg       0.86      0.85      0.85      2947
      weighted avg       0.86      0.86      0.85      2947



In [25]:
result = confusion_matrix(y_test,y_pred)
print('Confusion matrix kNN:')
print(result)

Confusion matrix kNN:
[[537   0   0   0   0   0]
 [  0 343 148   0   0   0]
 [  0  43 489   0   0   0]
 [  0   0   0 461  30   5]
 [  0   0   0  27 348  45]
 [  0   0   0  43  85 343]]
